In [6]:
# ---------------------------------------------------------------------------
# Phase 0: Setup & Configuration
# ---------------------------------------------------------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

print("--- Phase 0: Setup & Configuration Complete ---")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Seaborn version: {sns.__version__}")
print("-" * 70)

# Define File Paths (as provided by you)
sentiment_file_path = r"C:\Users\shiva\Downloads\fear_greed_index.csv"
trader_file_path = r"C:\Users\shiva\Downloads\historical_data.csv"

# ---------------------------------------------------------------------------
# Phase 1: Data Loading
# ---------------------------------------------------------------------------
print("\n--- Phase 1: Data Loading ---")
try:
    sentiment_df_raw = pd.read_csv(sentiment_file_path)
    print(f"Successfully loaded sentiment data from: {sentiment_file_path}")
    print("Sentiment Data (Raw) Head:\n", sentiment_df_raw.head())
    # sentiment_df_raw.info() # Uncomment for detailed info
except Exception as e:
    print(f"ERROR loading sentiment data: {e}")
    sentiment_df_raw = pd.DataFrame() # Empty df if load fails

try:
    trader_df_raw = pd.read_csv(trader_file_path)
    print(f"\nSuccessfully loaded trader data from: {trader_file_path}")
    print("Trader Data (Raw) Head:\n", trader_df_raw.head())
    # trader_df_raw.info() # Uncomment for detailed info
except Exception as e:
    print(f"ERROR loading trader data: {e}")
    trader_df_raw = pd.DataFrame() # Empty df if load fails

if sentiment_df_raw.empty or trader_df_raw.empty:
    print("\nERROR: One or both data files failed to load or are empty. Exiting.")
    raise SystemExit("Data loading failed.")
print("-" * 70)

# ---------------------------------------------------------------------------
# Phase 2: Data Preprocessing
# ---------------------------------------------------------------------------
print("\n--- Phase 2: Data Preprocessing ---")

# --- 2.1 Preprocessing Sentiment Data ---
print("\n--- 2.1 Preprocessing Sentiment Data ---")
sentiment_df = sentiment_df_raw.copy()

# Column names based on your previous KeyError output for sentiment data
# Available columns: ['timestamp', 'value', 'classification', 'date']
original_sent_date_col = 'date' # Or 'timestamp' - assuming 'date' is YYYY-MM-DD
original_sent_classification_col = 'classification'
original_sent_value_col = 'value' # Numeric score (0-100)

# Validate and rename sentiment columns
if original_sent_date_col not in sentiment_df.columns:
    raise KeyError(f"Sentiment date column '{original_sent_date_col}' not found.")
sentiment_df[original_sent_date_col] = pd.to_datetime(sentiment_df[original_sent_date_col])
sentiment_df = sentiment_df.set_index(original_sent_date_col)

rename_map_sentiment = {}
if original_sent_classification_col in sentiment_df.columns:
    rename_map_sentiment[original_sent_classification_col] = 'Classification'
if original_sent_value_col in sentiment_df.columns:
    rename_map_sentiment[original_sent_value_col] = 'SentimentValueRaw' # Raw numeric score
sentiment_df.rename(columns=rename_map_sentiment, inplace=True)

print("Sentiment Data (Processed) Head:\n", sentiment_df.head())
# sentiment_df.info()

# --- 2.2 Preprocessing Trader Data ---
print("\n--- 2.2 Preprocessing Trader Data ---")
trader_df = trader_df_raw.copy()

# Column names based on your previous KeyError output for trader data
# Available: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side',
# 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash',
# 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
original_trader_time_col = 'Timestamp' # Or 'Timestamp IST'
original_trader_symbol_col = 'Coin'
original_trader_pnl_col = 'Closed PnL'
original_trader_exec_price_col = 'Execution Price'
original_trader_size_col = 'Size Tokens' # Using token size for asset quantity
original_trader_side_col = 'Side'

# Columns NOT found in your trader data's available list, so set to None:
original_trader_event_col = None # No direct 'event' type column like 'fill', 'liquidation'
original_trader_leverage_col = None # No 'leverage' column

trader_rename_map = {}
if original_trader_time_col in trader_df.columns: trader_rename_map[original_trader_time_col] = 'time'
if original_trader_symbol_col in trader_df.columns: trader_rename_map[original_trader_symbol_col] = 'symbol'
if original_trader_pnl_col in trader_df.columns: trader_rename_map[original_trader_pnl_col] = 'closedPnL'
if original_trader_exec_price_col in trader_df.columns: trader_rename_map[original_trader_exec_price_col] = 'execution_price'
if original_trader_size_col in trader_df.columns: trader_rename_map[original_trader_size_col] = 'size'
if original_trader_side_col in trader_df.columns: trader_rename_map[original_trader_side_col] = 'side'

# Handle missing 'event' and 'leverage'
has_event_column = False
if original_trader_event_col and original_trader_event_col in trader_df.columns:
    trader_rename_map[original_trader_event_col] = 'event'
    has_event_column = True
else:
    print("INFO: Meaningful 'event' column (e.g., for liquidations) not found in trader data. Event-specific analysis will be limited.")
    trader_df['event'] = 'trade_occurrence' # Placeholder if needed by structure

has_leverage_column = False
if original_trader_leverage_col and original_trader_leverage_col in trader_df.columns:
    trader_rename_map[original_trader_leverage_col] = 'leverage'
    has_leverage_column = True
else:
    print("INFO: 'Leverage' column not found in trader data. Leverage-specific analysis will be skipped.")

trader_df.rename(columns=trader_rename_map, inplace=True)

# Validate essential columns after renaming
essential_cols = ['time', 'symbol', 'closedPnL', 'execution_price', 'size', 'side']
for col in essential_cols:
    if col not in trader_df.columns:
        raise KeyError(f"Trader data missing essential column '{col}' after renaming.")

# Convert 'time' column
try:
    trader_df['time'] = pd.to_datetime(trader_df['time'])
except Exception: # Broad exception for various parsing issues
    try:
        print("Attempting to parse trader 'time' column as Unix timestamp (ms)...")
        trader_df['time'] = pd.to_datetime(trader_df['time'], unit='ms')
    except Exception as e_time:
        print(f"ERROR: Failed to parse trader 'time' column: {e_time}")
        raise

# Filter for BTC trades
btc_symbol_pattern = 'BTC' # Adjust if your BTC symbol is more specific (e.g., 'BTC-PERP')
trader_df_btc = trader_df[trader_df['symbol'].astype(str).str.contains(btc_symbol_pattern, case=False, na=False)].copy()
print(f"Total trader records: {len(trader_df)}, BTC records: {len(trader_df_btc)}")
if trader_df_btc.empty:
    print("WARNING: No BTC trades found after filtering. Analysis will be on an empty dataset.")

# Convert PnL, size, price to numeric; coercing errors
trader_df_btc['closedPnL'] = pd.to_numeric(trader_df_btc['closedPnL'], errors='coerce')
trader_df_btc['size'] = pd.to_numeric(trader_df_btc['size'], errors='coerce')
trader_df_btc['execution_price'] = pd.to_numeric(trader_df_btc['execution_price'], errors='coerce')
if has_leverage_column and 'leverage' in trader_df_btc.columns: # Only if leverage column exists
    trader_df_btc['leverage'] = pd.to_numeric(trader_df_btc['leverage'], errors='coerce')

# Drop rows with NaN in critical numeric columns or 'side'
cols_to_check_na = ['closedPnL', 'size', 'execution_price', 'side']
if has_leverage_column and 'leverage' in trader_df_btc.columns:
    cols_to_check_na.append('leverage')

initial_btc_rows = len(trader_df_btc)
trader_df_btc.dropna(subset=[col for col in cols_to_check_na if col in trader_df_btc.columns], inplace=True)
print(f"BTC rows after NaN drop in critical columns: {len(trader_df_btc)} (removed {initial_btc_rows - len(trader_df_btc)})")

print("Trader Data (BTC, Processed) Head:\n", trader_df_btc.head())
# trader_df_btc.info()
print("-" * 70)

# ---------------------------------------------------------------------------
# Phase 3: Feature Engineering
# ---------------------------------------------------------------------------
print("\n--- Phase 3: Feature Engineering ---")

# --- 3.1 Sentiment Feature Engineering ---
if 'SentimentValueRaw' in sentiment_df.columns:
    sentiment_df['SentimentScore'] = sentiment_df['SentimentValueRaw'] # Using the direct numeric score
    print("Used 'SentimentValueRaw' for 'SentimentScore'.")
else:
    # Fallback to mapping text if numeric score isn't available/reliable
    print("Warning: 'SentimentValueRaw' not found. Attempting to map 'Classification' text for 'SentimentScore'.")
    sentiment_mapping = {'Extreme Fear': 1, 'Fear': 2, 'Neutral': 3, 'Greed': 4, 'Extreme Greed': 5}
    if 'Classification' in sentiment_df.columns:
        sentiment_df['SentimentScore'] = sentiment_df['Classification'].map(sentiment_mapping)
    else:
        print("ERROR: Neither 'SentimentValueRaw' nor 'Classification' found for SentimentScore creation.")
        sentiment_df['SentimentScore'] = np.nan # Placeholder

if 'Classification' in sentiment_df.columns:
    def get_broad_sentiment(text_classification):
        if pd.isna(text_classification): return 'Neutral'
        text_classification = str(text_classification).lower()
        if 'extreme fear' in text_classification or 'fear' in text_classification: return 'Fear'
        if 'extreme greed' in text_classification or 'greed' in text_classification: return 'Greed'
        return 'Neutral'
    sentiment_df['BroadSentiment'] = sentiment_df['Classification'].apply(get_broad_sentiment)
else:
    print("Warning: 'Classification' column not available for 'BroadSentiment'. Setting to 'Neutral'.")
    sentiment_df['BroadSentiment'] = 'Neutral'

print("Sentiment Data with Features (head):\n", sentiment_df.head())

# --- 3.2 Trader Feature Engineering ---
if not trader_df_btc.empty:
    trader_df_btc['trade_date'] = trader_df_btc['time'].dt.normalize() # Date part for merging
    trader_df_btc['is_profit'] = trader_df_btc['closedPnL'] > 0
    print("Trader BTC Data with Features (head):\n", trader_df_btc.head())
else:
    print("Trader BTC data is empty, skipping feature engineering for it.")
print("-" * 70)

# ---------------------------------------------------------------------------
# Phase 4: Data Merging
# ---------------------------------------------------------------------------
print("\n--- Phase 4: Data Merging ---")
if not trader_df_btc.empty and 'trade_date' in trader_df_btc.columns:
    merged_df = pd.merge(trader_df_btc, sentiment_df[['SentimentScore', 'BroadSentiment', 'Classification']], # Select specific columns
                         left_on='trade_date', right_index=True, how='left')
    print(f"Rows before merge: {len(trader_df_btc)}, Rows after merge: {len(merged_df)}")

    # Drop rows where sentiment data couldn't be merged
    merged_df.dropna(subset=['SentimentScore', 'BroadSentiment'], inplace=True)
    print(f"Rows after dropping non-merged sentiment: {len(merged_df)}")
    print("Merged Data (Head):\n", merged_df.head())
else:
    print("Trader BTC data is empty or 'trade_date' missing. Cannot merge. EDA will be skipped.")
    merged_df = pd.DataFrame() # Ensure merged_df exists as empty if merge fails
print("-" * 70)

# ---------------------------------------------------------------------------
# Phase 5: Exploratory Data Analysis (EDA)
# ---------------------------------------------------------------------------
print("\n--- Phase 5: Exploratory Data Analysis (EDA) ---")
if merged_df.empty:
    print("Merged DataFrame is empty. EDA cannot be performed.")
else:
    plt.style.use('seaborn-v0_8-darkgrid') # Set a pleasant style

    # 5.1 Overall PnL Distribution
    plt.figure(figsize=(10, 6))
    sns.histplot(merged_df['closedPnL'], kde=True, bins=50)
    plt.title('Distribution of Closed PnL (BTC Trades)')
    plt.xlabel('Closed PnL'); plt.ylabel('Frequency'); plt.show()
    print("PnL Stats:\n", merged_df['closedPnL'].describe())

    # 5.2 Sentiment Distribution on Traded Days
    plt.figure(figsize=(8, 5))
    merged_df['BroadSentiment'].value_counts(normalize=True).sort_index().plot(kind='bar', color=['red', 'lightgray', 'green'])
    plt.title('Sentiment Distribution on Traded Days')
    plt.ylabel('Proportion of Trades'); plt.xticks(rotation=0); plt.show()

    # 5.3 PnL by Sentiment
    plt.figure(figsize=(10, 6))
    sns.boxplot(data=merged_df, x='BroadSentiment', y='closedPnL', order=['Fear', 'Neutral', 'Greed'])
    plt.title('PnL Distribution by Broad Sentiment')
    # Consider y-axis limits if outliers are extreme: merged_df['closedPnL'].quantile([0.05, 0.95])
    plt.show()
    print("Average PnL by Sentiment:\n", merged_df.groupby('BroadSentiment')['closedPnL'].mean().reindex(['Fear', 'Neutral', 'Greed']))

    # 5.4 Win Rate by Sentiment
    win_rate_df = merged_df.groupby('BroadSentiment')['is_profit'].mean().reindex(['Fear', 'Neutral', 'Greed'])
    print("Win Rate by Sentiment:\n", win_rate_df)
    plt.figure(figsize=(8, 5))
    win_rate_df.plot(kind='bar', color=['red', 'lightgray', 'green'])
    plt.title('Win Rate by Broad Sentiment'); plt.ylabel('Win Rate'); plt.ylim(0,1); plt.xticks(rotation=0); plt.show()

    # 5.5 Leverage Analysis (Conditional)
    if has_leverage_column and 'leverage' in merged_df.columns:
        print("\n--- Leverage Analysis ---")
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=merged_df, x='BroadSentiment', y='leverage', order=['Fear', 'Neutral', 'Greed'])
        plt.title('Leverage Distribution by Sentiment'); plt.show()
        print("Average Leverage by Sentiment:\n", merged_df.groupby('BroadSentiment')['leverage'].mean().reindex(['Fear', 'Neutral', 'Greed']))

        plt.figure(figsize=(12, 7))
        sns.scatterplot(data=merged_df, x='leverage', y='closedPnL', hue='BroadSentiment', alpha=0.5,
                        palette={'Fear':'red', 'Neutral':'lightgray', 'Greed':'green'})
        plt.title('PnL vs. Leverage, Colored by Sentiment'); plt.axhline(0, color='k', linestyle='--', lw=0.8); plt.show()
    else:
        print("\nINFO: Leverage column not available. Skipping leverage-specific EDA.")

    # 5.6 Trading Activity (Size, Count) by Sentiment
    print("\n--- Trading Activity Analysis ---")
    activity_df = merged_df.groupby('BroadSentiment').agg(
        num_trades=('symbol', 'count'),
        avg_trade_size=('size', 'mean'),
        total_trade_size=('size', 'sum')
    ).reindex(['Fear', 'Neutral', 'Greed'])
    print("Trading Activity by Sentiment:\n", activity_df)
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    activity_df['num_trades'].plot(kind='bar', ax=axes[0], title='Number of Trades by Sentiment', color=['red', 'lightgray', 'green'])
    activity_df['avg_trade_size'].plot(kind='bar', ax=axes[1], title='Average Trade Size by Sentiment', color=['red', 'lightgray', 'green'])
    plt.tight_layout(); plt.show()

    # 5.7 Trade Side (Buy/Sell) Performance by Sentiment
    print("\n--- Trade Side Performance Analysis ---")
    side_perf_df = merged_df.groupby(['BroadSentiment', 'side'])['closedPnL'].agg(['mean', 'count']).unstack()
    print("PnL by Sentiment and Trade Side:\n", side_perf_df)
    if not side_perf_df.empty and ('mean', 'buy') in side_perf_df.columns and ('mean', 'sell') in side_perf_df.columns:
         side_perf_df[('mean')].reindex(['Fear', 'Neutral', 'Greed']).plot(kind='bar', figsize=(10,6))
         plt.title('Average PnL by Sentiment and Trade Side'); plt.ylabel('Average PnL'); plt.xticks(rotation=0); plt.show()
    else:
        print("Not enough data diversity for side performance plot.")
    
    # 5.8 Event Analysis (Limited due to missing specific event column)
    if 'event' in merged_df.columns and merged_df['event'].nunique() > 1 and merged_df['event'].iloc[0] != 'trade_occurrence': # Check if not just placeholder
        print("\n--- Event Analysis (if meaningful events exist) ---")
        print("Event Counts:\n", merged_df['event'].value_counts())
        # If you identify a specific event string like 'liquidation' in your actual data (not the placeholder):
        # liquidation_str = 'YOUR_LIQUIDATION_EVENT_STRING'
        # liquidations = merged_df[merged_df['event'] == liquidation_str]
        # if not liquidations.empty:
        #     liquidations.groupby('BroadSentiment').size().reindex(['Fear', 'Neutral', 'Greed']).plot(kind='bar', title='Liquidations by Sentiment')
        #     plt.show()
    else:
        print("\nINFO: No distinct event types found or using placeholder 'event'. Detailed event analysis skipped.")

print("-" * 70)

# ---------------------------------------------------------------------------
# Phase 6: Insight Generation & Strategy Derivation
# ---------------------------------------------------------------------------
print("\n--- Phase 6: Insight Generation & Strategy Derivation ---")
if merged_df.empty:
    print("No data available for insights as Merged DataFrame is empty.")
else:
    print("\nKey Observations (based on this specific dataset analysis):")
    # General PnL
    overall_avg_pnl = merged_df['closedPnL'].mean()
    print(f"- Overall average PnL per trade: {overall_avg_pnl:.2f}.")

    # PnL by Sentiment
    avg_pnl_fear = merged_df[merged_df['BroadSentiment'] == 'Fear']['closedPnL'].mean()
    avg_pnl_neutral = merged_df[merged_df['BroadSentiment'] == 'Neutral']['closedPnL'].mean()
    avg_pnl_greed = merged_df[merged_df['BroadSentiment'] == 'Greed']['closedPnL'].mean()
    print(f"- Average PnL during Fear: {avg_pnl_fear:.2f}, Neutral: {avg_pnl_neutral:.2f}, Greed: {avg_pnl_greed:.2f}.")
    if avg_pnl_fear > avg_pnl_greed and avg_pnl_fear > avg_pnl_neutral:
        print("  > Trades during 'Fear' periods showed the highest average PnL.")
    elif avg_pnl_greed > avg_pnl_fear and avg_pnl_greed > avg_pnl_neutral:
        print("  > Trades during 'Greed' periods showed the highest average PnL.")

    # Win Rate by Sentiment
    win_rate_fear = merged_df[merged_df['BroadSentiment'] == 'Fear']['is_profit'].mean()
    win_rate_greed = merged_df[merged_df['BroadSentiment'] == 'Greed']['is_profit'].mean()
    print(f"- Win rate during Fear: {win_rate_fear:.2%}, Greed: {win_rate_greed:.2%}.")
    if win_rate_fear > win_rate_greed:
        print("  > Win rates appear higher during 'Fear' periods.")
    elif win_rate_greed > win_rate_fear:
         print("  > Win rates appear higher during 'Greed' periods.")

    # Leverage Insights (if available)
    if has_leverage_column and 'leverage' in merged_df.columns:
        avg_leverage_fear = merged_df[merged_df['BroadSentiment'] == 'Fear']['leverage'].mean()
        avg_leverage_greed = merged_df[merged_df['BroadSentiment'] == 'Greed']['leverage'].mean()
        print(f"- Average leverage during Fear: {avg_leverage_fear:.2f}x, Greed: {avg_leverage_greed:.2f}x.")
        # Further insights would come from visually inspecting PnL vs Leverage plot.
        print("  > (Visual inspection needed for PnL vs Leverage correlation by sentiment).")
    else:
        print("- Leverage data was not available for detailed insights.")

    print("\nPotential Strategy Considerations (Illustrative - Requires Rigorous Backtesting):")
    print("1. Sentiment-Biased Direction: If analysis consistently shows, for example, that long positions perform better in 'Greed' and shorts in 'Fear', traders could consider aligning their primary trade direction with strong sentiment signals.")
    print("2. Contrarian Opportunities: If periods of 'Extreme Fear' (low SentimentScore) are often followed by reversals, this could signal contrarian buying opportunities, especially if win rates improve then.")
    print("3. Risk Adjustment by Sentiment: If volatility or losses are notably higher during 'Fear' (even without leverage data), traders might consider reducing position sizes or tightening stop-losses during such periods.")
    print("4. Activity Monitoring: If trade volume or frequency significantly changes with sentiment, this could indicate broader market participation shifts that might precede price movements.")

    print("\nLimitations & Next Steps:")
    print("- The analysis is based on historical data and does not guarantee future performance.")
    if not has_leverage_column:
        print("- Lack of 'leverage' data limits understanding of risk-taking behavior.")
    if not has_event_column or (has_event_column and merged_df['event'].iloc[0] == 'trade_occurrence'):
        print("- Lack of specific 'event' data (e.g., liquidations) limits analysis of extreme market events.")
    print("- Further analysis could involve: lagged sentiment effects, trader segmentation (if 'Account' data were used), and statistical significance testing of observed differences.")

print("\n--- Analysis Complete ---")

--- Phase 0: Setup & Configuration Complete ---
Pandas version: 2.2.3
NumPy version: 2.2.5
Seaborn version: 0.13.2
----------------------------------------------------------------------

--- Phase 1: Data Loading ---
Successfully loaded sentiment data from: C:\Users\shiva\Downloads\fear_greed_index.csv
Sentiment Data (Raw) Head:
     timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03
3  1517722200     24   Extreme Fear  2018-02-04
4  1517808600     11   Extreme Fear  2018-02-05

Successfully loaded trader data from: C:\Users\shiva\Downloads\historical_data.csv
Trader Data (Raw) Head:
                                       Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107    